In [ ]:
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

matplotlib.rcParams.update({'font.size': 18})

In [ ]:
NUM_BINS = 10
# plot_colors = [mcolors.BASE_COLORS[x] for x in mcolors.BASE_COLORS.keys()]
plot_colors = [mcolors.TABLEAU_COLORS[x] for x in mcolors.TABLEAU_COLORS.keys()]*2
line_styles = ["--", "-", ":", "-.", (0, (3, 5, 1, 5, 1, 5)), (0, (3, 1, 1, 1, 1, 1))]*2

In [ ]:
# >> TODO: UPDATE HERE <<
DATASET = "Atari"
MODEL_NAME = "NatureDQNNetwork"
NUM_EXPERIENCES = 5

# all combinations of training
run2name = {
    # "j_nst_npp_v1" : ("JointTraining_NoSelfTraining_NoPostProcessing1", "./logs/Atari"),
    # "j_nst_npp_v2" : ("JointTraining_NoSelfTraining_NoPostProcessing2", "./logs/Atari"),
    # "j_nst_npp_v3" : ("JointTraining_NoSelfTraining_NoPostProcessing3", "./logs/Atari"),
    # "j_st_npp_00075_v1" : ("JointTraining_SelfTraining_0.0075_NoPostProcessing1", "./logs/Atari_ST"),
    # "j_st_npp_00075_v2" : ("JointTraining_SelfTraining_0.0075_NoPostProcessing2", "./logs/Atari_ST"),
    # "j_st_npp_00075_v3" : ("JointTraining_SelfTraining_0.0075_NoPostProcessing3", "./logs/Atari_ST"),
    # "j_pp_ts_v0" : ("JointTraining_NoSelfTraining_PostProcessing_TemperatureScaling1", "./logs/Atari_TS2"),
    # "j_pp_ts_v1" : ("JointTraining_NoSelfTraining_PostProcessing_TemperatureScaling2", "./logs/Atari_TS2"),
    # "j_pp_ts_v2" : ("JointTraining_NoSelfTraining_PostProcessing_TemperatureScaling3", "./logs/Atari_TS2"),
    # "j_pp_vs_v1" : ("JointTraining_NoSelfTraining_PostProcessing_VectorScaling1", "./logs/Atari_VS2"),
    # "j_pp_vs_v2" : ("JointTraining_NoSelfTraining_PostProcessing_VectorScaling2", "./logs/Atari_VS2"),
    # "j_pp_vs_v3" : ("JointTraining_NoSelfTraining_PostProcessing_VectorScaling3", "./logs/Atari_VS2"),
    # "j_pp_ms_v1" : ("JointTraining_NoSelfTraining_PostProcessing_MatrixScaling1", "./logs/Atari_MS"),
    # "j_pp_ms_v2" : ("JointTraining_NoSelfTraining_PostProcessing_MatrixScaling2", "./logs/Atari_MS"),
    # "j_pp_ms_v3" : ("JointTraining_NoSelfTraining_PostProcessing_MatrixScaling3", "./logs/Atari_MS"),

    # "r_nst_npp_v1" : ("Replay_NoSelfTraining_NoPostProcessing1", "./logs/Atari_R"),
    # "r_nst_npp_v2" : ("Replay_NoSelfTraining_NoPostProcessing2", "./logs/Atari_R"),
    # "r_nst_npp_v3" : ("Replay_NoSelfTraining_NoPostProcessing3", "./logs/Atari_R"),
    # "r_st_npp_00025_v1" : ("Replay_SelfTraining_0.0025_NoPostProcessing4", "./logs/Atari_R_ST"),
    # "r_st_npp_00025_v2" : ("Replay_SelfTraining_0.0025_NoPostProcessing2", "./logs/Atari_R_ST"),
    # "r_st_npp_00025_v3" : ("Replay_SelfTraining_0.0025_NoPostProcessing3", "./logs/Atari_R_ST"),
    # "r_pp_ts_v0" : ("Replay_NoSelfTraining_PostProcessing_TemperatureScaling1", "./logs/Atari_R_TS"),
    # "r_pp_ts_v1" : ("Replay_NoSelfTraining_PostProcessing_TemperatureScaling2", "./logs/Atari_R_TS"),
    # "r_pp_ts_v2" : ("Replay_NoSelfTraining_PostProcessing_TemperatureScaling3", "./logs/Atari_R_TS"),
    # "r_pp_vs_v1" : ("Replay_NoSelfTraining_PostProcessing_VectorScaling1", "./logs/Atari_R_VS2"),
    # "r_pp_vs_v2" : ("Replay_NoSelfTraining_PostProcessing_VectorScaling2", "./logs/Atari_R_VS2"),
    # "r_pp_vs_v3" : ("Replay_NoSelfTraining_PostProcessing_VectorScaling3", "./logs/Atari_R_VS2"),
    # "r_pp_ms_v1" : ("Replay_NoSelfTraining_PostProcessing_MatrixScaling1", "./logs/Atari_R_MS2"),
    # "r_pp_ms_v2" : ("Replay_NoSelfTraining_PostProcessing_MatrixScaling2", "./logs/Atari_R_MS2"),
    # "r_pp_ms_v3" : ("Replay_NoSelfTraining_PostProcessing_MatrixScaling3", "./logs/Atari_R_MS2"),
    # "r_pp_ts_md_v0" : ("Replay_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData1", "./logs/Atari_R_TS_MD"),
    # "r_pp_ts_md_v1" : ("Replay_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData2", "./logs/Atari_R_TS_MD"),
    # "r_pp_ts_md_v2" : ("Replay_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData3", "./logs/Atari_R_TS_MD"),
    # "r_pp_vs_md_v1" : ("Replay_NoSelfTraining_PostProcessing_VectorScaling_MixedData1", "./logs/Atari_R_VS_MD"),
    # "r_pp_vs_md_v2" : ("Replay_NoSelfTraining_PostProcessing_VectorScaling_MixedData2", "./logs/Atari_R_VS_MD"),
    # "r_pp_vs_md_v3" : ("Replay_NoSelfTraining_PostProcessing_VectorScaling_MixedData3", "./logs/Atari_R_VS_MD"),
    # "r_pp_ms_md_v1" : ("Replay_NoSelfTraining_PostProcessing_MatrixScaling_MixedData1", "./logs/Atari_R_MS_MD"),
    # "r_pp_ms_md_v2" : ("Replay_NoSelfTraining_PostProcessing_MatrixScaling_MixedData2", "./logs/Atari_R_MS_MD"),
    # "r_pp_ms_md_v3" : ("Replay_NoSelfTraining_PostProcessing_MatrixScaling_MixedData3", "./logs/Atari_R_MS_MD"),

    # "n_nst_npp_v1" : ("Naive_NoSelfTraining_NoPostProcessing1", "./logs/Atari_N"),
    # "n_nst_npp_v2" : ("Naive_NoSelfTraining_NoPostProcessing2", "./logs/Atari_N"),
    # "n_nst_npp_v3" : ("Naive_NoSelfTraining_NoPostProcessing3", "./logs/Atari_N"),  
    # "n_st_npp_0005_v1" : ("Naive_SelfTraining_0.005_NoPostProcessing1", "./logs/Atari_N_ST"), 
    # "n_st_npp_0005_v2" : ("Naive_SelfTraining_0.005_NoPostProcessing2", "./logs/Atari_N_ST"), 
    # "n_st_npp_0005_v3" : ("Naive_SelfTraining_0.005_NoPostProcessing3", "./logs/Atari_N_ST"), 
    # "n_pp_ts_v0" : ("Naive_NoSelfTraining_PostProcessing_TemperatureScaling1", "./logs/Atari_N_TS2"), 
    # "n_pp_ts_v1" : ("Naive_NoSelfTraining_PostProcessing_TemperatureScaling2", "./logs/Atari_N_TS2"), 
    # "n_pp_ts_v2" : ("Naive_NoSelfTraining_PostProcessing_TemperatureScaling3", "./logs/Atari_N_TS2"), 
    # "n_pp_vs_v1" : ("Naive_NoSelfTraining_PostProcessing_VectorScaling1", "./logs/Atari_N_VS2"), 
    # "n_pp_vs_v2" : ("Naive_NoSelfTraining_PostProcessing_VectorScaling2", "./logs/Atari_N_VS2"), 
    # "n_pp_vs_v3" : ("Naive_NoSelfTraining_PostProcessing_VectorScaling3", "./logs/Atari_N_VS2"), 
    # "n_pp_ms_v1" : ("Naive_NoSelfTraining_PostProcessing_MatrixScaling1", "./logs/Atari_N_MS2"), 
    # "n_pp_ms_v2" : ("Naive_NoSelfTraining_PostProcessing_MatrixScaling2", "./logs/Atari_N_MS2"), 
    # "n_pp_ms_v3" : ("Naive_NoSelfTraining_PostProcessing_MatrixScaling3", "./logs/Atari_N_MS2"), 
    # "n_pp_ts_md_v0" : ("Naive_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData1", "./logs/Atari_N_TS_MD"), 
    # "n_pp_ts_md_v1" : ("Naive_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData2", "./logs/Atari_N_TS_MD"), 
    # "n_pp_ts_md_v2" : ("Naive_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData3", "./logs/Atari_N_TS_MD"), 
    # "n_pp_vs_md_v1" : ("Naive_NoSelfTraining_PostProcessing_VectorScaling_MixedData1", "./logs/Atari_N_VS_MD"),
    # "n_pp_vs_md_v2" : ("Naive_NoSelfTraining_PostProcessing_VectorScaling_MixedData2", "./logs/Atari_N_VS_MD"),
    # "n_pp_vs_md_v3" : ("Naive_NoSelfTraining_PostProcessing_VectorScaling_MixedData3", "./logs/Atari_N_VS_MD"),
    # "n_pp_ms_md_v1" : ("Naive_NoSelfTraining_PostProcessing_MatrixScaling_MixedData1", "./logs/Atari_N_MS_MD"),
    # "n_pp_ms_md_v2" : ("Naive_NoSelfTraining_PostProcessing_MatrixScaling_MixedData2", "./logs/Atari_N_MS_MD"),
    # "n_pp_ms_md_v3" : ("Naive_NoSelfTraining_PostProcessing_MatrixScaling_MixedData3", "./logs/Atari_N_MS_MD"),

    # "d_nst_npp_v19" : ("DER_NoSelfTraining_NoPostProcessing19", "./logs/Atari_D_HP"),
    # "d_nst_npp_v20" : ("DER_NoSelfTraining_NoPostProcessing20", "./logs/Atari_D_HP"),
    # "d_nst_npp_v21" : ("DER_NoSelfTraining_NoPostProcessing21", "./logs/Atari_D_HP"),
    # "d_st_npp_0025" : ("DER_SelfTraining_0.025_NoPostProcessing1", "./logs/Atari_D_ST"),
    # "d_st_npp_0025_2" : ("DER_SelfTraining_0.025_NoPostProcessing2", "./logs/Atari_D_ST"),
    # "d_st_npp_0025_3" : ("DER_SelfTraining_0.025_NoPostProcessing3", "./logs/Atari_D_ST"),
    # "d_pp_ts_v0" : ("DER_NoSelfTraining_PostProcessing_TemperatureScaling1", "./logs/Atari_D_TS"), 
    # "d_pp_ts_v1" : ("DER_NoSelfTraining_PostProcessing_TemperatureScaling2", "./logs/Atari_D_TS"), 
    # "d_pp_ts_v2" : ("DER_NoSelfTraining_PostProcessing_TemperatureScaling3", "./logs/Atari_D_TS"), 
    # "d_pp_vs_v1" : ("DER_NoSelfTraining_PostProcessing_VectorScaling1", "./logs/Atari_D_VS"), 
    # "d_pp_vs_v2" : ("DER_NoSelfTraining_PostProcessing_VectorScaling2", "./logs/Atari_D_VS"), 
    # "d_pp_vs_v3" : ("DER_NoSelfTraining_PostProcessing_VectorScaling3", "./logs/Atari_D_VS"), 
    # "d_pp_ms_v1" : ("DER_NoSelfTraining_PostProcessing_MatrixScaling1", "./logs/Atari_D_MS"), 
    # "d_pp_ms_v2" : ("DER_NoSelfTraining_PostProcessing_MatrixScaling2", "./logs/Atari_D_MS"), 
    # "d_pp_ms_v3" : ("DER_NoSelfTraining_PostProcessing_MatrixScaling3", "./logs/Atari_D_MS"), 
    # "d_pp_ts_md_v0" : ("DER_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData1", "./logs/Atari_D_TS_MD"), 
    # "d_pp_ts_md_v1" : ("DER_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData2", "./logs/Atari_D_TS_MD"), 
    # "d_pp_ts_md_v2" : ("DER_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData3", "./logs/Atari_D_TS_MD"), 
    # "d_pp_vs_md_v1" : ("DER_NoSelfTraining_PostProcessing_VectorScaling_MixedData1", "./logs/Atari_D_VS_MD"),
    # "d_pp_vs_md_v2" : ("DER_NoSelfTraining_PostProcessing_VectorScaling_MixedData2", "./logs/Atari_D_VS_MD"),
    # "d_pp_vs_md_v3" : ("DER_NoSelfTraining_PostProcessing_VectorScaling_MixedData3", "./logs/Atari_D_VS_MD"),
    # "d_pp_ms_md_v1" : ("DER_NoSelfTraining_PostProcessing_MatrixScaling_MixedData1", "./logs/Atari_D_MS_MD"),
    # "d_pp_ms_md_v2" : ("DER_NoSelfTraining_PostProcessing_MatrixScaling_MixedData2", "./logs/Atari_D_MS_MD"),
    # "d_pp_ms_md_v3" : ("DER_NoSelfTraining_PostProcessing_MatrixScaling_MixedData3", "./logs/Atari_D_MS_MD"),
}

In [ ]:
run2label = {
    "j_nst_npp_v1" : "Joint",
    "j_nst_npp_v2" : "Joint",
    "j_nst_npp_v3" : "Joint",
    "j_st_npp_00075_v1" : "Joint+HR",
    "j_st_npp_00075_v2" : "Joint+HR",
    "j_st_npp_00075_v3" : "Joint+HR",
    "j_pp_vs_v1" : "Joint+VS",
    "j_pp_vs_v2" : "Joint+VS",
    "j_pp_vs_v3" : "Joint+VS",
    "j_pp_ms_v1" : "Joint+MS",
    "j_pp_ms_v2" : "Joint+MS",
    "j_pp_ms_v3" : "Joint+MS",
    "j_pp_ts_v0" : "Joint+TS",
    "j_pp_ts_v1" : "Joint+TS",
    "j_pp_ts_v2" : "Joint+TS",

    "r_nst_npp_v1" : "Replay",
    "r_nst_npp_v2" : "Replay",
    "r_nst_npp_v3" : "Replay",
    "r_st_npp_00025_v1" : "Replay+HR",
    "r_st_npp_00025_v2" : "Replay+HR",
    "r_st_npp_00025_v3" : "Replay+HR",
    "r_pp_vs_v1" : "Replay+VS",
    "r_pp_vs_v2" : "Replay+VS",
    "r_pp_vs_v3" : "Replay+VS",
    "r_pp_ms_v1" : "Replay+MS",
    "r_pp_ms_v2" : "Replay+MS",
    "r_pp_ms_v3" : "Replay+MS",
    "r_pp_ts_v0" : "Replay+TS",
    "r_pp_ts_v1" : "Replay+TS",
    "r_pp_ts_v2" : "Replay+TS",
    "r_pp_vs_md_v1" : "Replay+VS+RC",
    "r_pp_vs_md_v2" : "Replay+VS+RC",
    "r_pp_vs_md_v3" : "Replay+VS+RC",
    "r_pp_ms_md_v1" : "Replay+MS+RC",
    "r_pp_ms_md_v2" : "Replay+MS+RC",
    "r_pp_ms_md_v3" : "Replay+MS+RC",
    "r_pp_ts_md_v0" : "Replay+TS+RC",
    "r_pp_ts_md_v1" : "Replay+TS+RC",
    "r_pp_ts_md_v2" : "Replay+TS+RC",
    
    "n_nst_npp_v1" : "Naive",
    "n_nst_npp_v2" : "Naive",
    "n_nst_npp_v3" : "Naive",
    "n_st_npp_001_v1" : "Naive+HR",
    "n_st_npp_001_v2" : "Naive+HR",
    "n_st_npp_001_v3" : "Naive+HR",
    "n_st_npp_0005_v1" : "Naive+HR",
    "n_st_npp_0005_v2" : "Naive+HR",
    "n_st_npp_0005_v3" : "Naive+HR",
    "n_pp_vs_v1" : "Naive+VS",
    "n_pp_vs_v2" : "Naive+VS",
    "n_pp_vs_v3" : "Naive+VS",
    "n_pp_ms_v1" : "Naive+MS",
    "n_pp_ms_v2" : "Naive+MS",
    "n_pp_ms_v3" : "Naive+MS",
    "n_pp_ts_v0" : "Naive+TS",
    "n_pp_ts_v1" : "Naive+TS",
    "n_pp_ts_v2" : "Naive+TS",
    "n_pp_vs_md_v1" : "Naive+VS+RC",
    "n_pp_vs_md_v2" : "Naive+VS+RC",
    "n_pp_vs_md_v3" : "Naive+VS+RC",
    "n_pp_ms_md_v1" : "Naive+MS+RC",
    "n_pp_ms_md_v2" : "Naive+MS+RC",
    "n_pp_ms_md_v3" : "Naive+MS+RC",
    "n_pp_ts_md_v0" : "Naive+TS+RC",
    "n_pp_ts_md_v1" : "Naive+TS+RC",
    "n_pp_ts_md_v2" : "Naive+TS+RC",

    "d_nst_npp_v19" : "DER",
    "d_nst_npp_v20" : "DER",
    "d_nst_npp_v21" : "DER",
    "d_st_npp_0025" : "DER+HR",
    "d_st_npp_0025_2" : "DER+HR",
    "d_st_npp_0025_3" : "DER+HR",
    "d_pp_ts_v0" : "DER+TS",
    "d_pp_ts_v1" : "DER+TS",
    "d_pp_ts_v2" : "DER+TS",
    "d_pp_vs_v1" : "DER+VS",
    "d_pp_vs_v2" : "DER+VS",
    "d_pp_vs_v3" : "DER+VS",
    "d_pp_ms_v1" : "DER+MS",
    "d_pp_ms_v2" : "DER+MS",
    "d_pp_ms_v3" : "DER+MS",
    "d_pp_ts_md_v0" : "DER+TS+RC",
    "d_pp_ts_md_v1" : "DER+TS+RC",
    "d_pp_ts_md_v2" : "DER+TS+RC",
    "d_pp_vs_md_v1" : "DER+VS+RC",
    "d_pp_vs_md_v2" : "DER+VS+RC",
    "d_pp_vs_md_v3" : "DER+VS+RC",
    "d_pp_ms_md_v1" : "DER+MS+RC",
    "d_pp_ms_md_v2" : "DER+MS+RC",
    "d_pp_ms_md_v3" : "DER+MS+RC",
}

In [ ]:
running_accuracy = []
running_ece = []
final_accuracy = []
final_ece = []
bins = None
ece_hist_vals = []

for k, (name, path) in run2name.items():
    print(f">> {name} <<")
    with open(f"{path}/{DATASET}_{MODEL_NAME}_{name}_dict", "rb") as file:
        data = pickle.load(file)

        # print("\n---- ACCURACY ----")

        metric_str = "Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp"
        m = []
        for i in range(len(data)):
            cur_exp_dict = data[i]
            cur_exp_acc = 0
            # compute the average over the experiences trained so far (i)
            for j in range(i+1):
                # print(i, j, metric_str + f"{j:03d}", cur_exp_dict[metric_str + f"{j:03d}"])
                cur_exp_acc += cur_exp_dict[metric_str + f"{j:03d}"]
            m.append(cur_exp_acc/(i+1))
        
        # duplicate for JointTraining
        if len(m) < NUM_EXPERIENCES:
            m = m*NUM_EXPERIENCES
        running_accuracy.append((k, m))
        final_accuracy.append((k, running_accuracy[-1][-1][-1]))
        # print(k, m, running_accuracy[-1][-1][-1])

        # print("\n---- ECE ----")

        metric_str = "ECE_Exp/eval_phase/test_stream/Task000/Exp"
        m = []
        for i in range(len(data)):
            cur_exp_dict = data[i]
            cur_exp_ece = 0
            # compute the average over the experiences trained so far (i)
            for j in range(i+1):
                # print(i, j, metric_str + f"{j:03d}", cur_exp_dict[metric_str + f"{j:03d}"])
                cur_exp_ece += cur_exp_dict[metric_str + f"{j:03d}"]
            # m.append(cur_exp_ece/(i+1))
            m.append((cur_exp_ece/(i+1))*100)
        
        # duplicate for JointTraining
        if len(m) < NUM_EXPERIENCES:
            m = m*NUM_EXPERIENCES
        running_ece.append((k, m))
        final_ece.append((k, running_ece[-1][-1][-1]))
        # print(k, m, running_ece[-1][-1][-1])

        # print("\n---- ECE HISTOGRAMS ----")

        metric_str = "ExpECEHistogram/eval_phase/test_stream/Exp"
        m = []
        i = -1 # after last experience
        cur_exp_dict = data[i]
        for j in range(NUM_EXPERIENCES):
            # print(i, j, metric_str + f"{j:03d}", cur_exp_dict[metric_str + f"{j:03d}"])
            fig = cur_exp_dict[metric_str + f"{j:03d}"]
            axes_list = fig.get_axes()
            for ax in axes_list:
                for line in ax.get_lines()[-1:]:
                    x_data = line.get_xdata()
                    y_data = line.get_ydata()
                    # print({'x': x_data, 'y': y_data})
                    if bins is None:
                        bins = x_data
                    m.append(y_data)
        # print(bins, m)
        bin_vals = []
        for i in range(len(bins)):
            x = []
            for j in range(len(m)):
                # print(j, i, m[j][i])
                x.append(m[j][i])
            mean = np.mean(x)
            std = np.std(x)
            bin_vals.append((mean, std))
        # print(k, bin_vals)
        ece_hist_vals.append((k, bin_vals))


In [ ]:
acc_data = {}
acc_plot_data = {}
for i, (name, vals) in enumerate(running_accuracy):
    if run2label[name] not in acc_data:
        acc_data[run2label[name]] = [[] for _ in range(NUM_EXPERIENCES)]
    for i, val in enumerate(vals):
        acc_data[run2label[name]][i].append(val)
for i, (name, data) in enumerate(acc_data.items()):
    for values in data:
        mean = np.mean(values)
        std = np.std(values)
        if name not in acc_plot_data:
            acc_plot_data[name] = [[], []]
        acc_plot_data[name][0].append(mean)
        acc_plot_data[name][1].append(std)

ece_data = {}
ece_plot_data = {}
for i, (name, vals) in enumerate(running_ece):
    if run2label[name] not in ece_data:
        ece_data[run2label[name]] = [[] for _ in range(NUM_EXPERIENCES)]
    for i, val in enumerate(vals):
        ece_data[run2label[name]][i].append(val)
for i, (name, data) in enumerate(ece_data.items()):
    for values in data:
        mean = np.mean(values)
        std = np.std(values)
        if name not in ece_plot_data:
            ece_plot_data[name] = [[], []]
        ece_plot_data[name][0].append(mean)
        ece_plot_data[name][1].append(std)

rel_data = {}
rel_plot_data = {}
for i, (name, vals) in enumerate(ece_hist_vals):
    # print(i, name, vals)
    if run2label[name] not in rel_data:
        rel_data[run2label[name]] = [[] for _ in range(NUM_BINS)]
    for i, val in enumerate(vals):
        rel_data[run2label[name]][i].append(val[0])
for i, (name, data) in enumerate(rel_data.items()):
    # print(i, name, data)
    for values in data:
        mean = np.mean(values)
        std = np.std(values)
        if name not in rel_plot_data:
            rel_plot_data[name] = [[], []]
        rel_plot_data[name][0].append(mean)
        rel_plot_data[name][1].append(std)

In [ ]:
# PLOT 1: Average accuracy on all experiences after training on exp j

plt.figure(figsize=(7, 6))
x_axis = list(range(1, NUM_EXPERIENCES+1))
for i, (name, (means, stds)) in enumerate(acc_plot_data.items()):
    # print(i, name, means, stds)
    plt.plot(x_axis, means, color=plot_colors[i+1], label=name, linestyle=line_styles[i+1])
    l = [means[i] - stds[i] for i in range(len(means))]
    u = [means[i] + stds[i] for i in range(len(means))]
    plt.fill_between(x_axis, l, u, alpha=0.2, color=plot_colors[i+1])
# plt.title(f'{DATASET} - Accuracy')
plt.xlabel('#Trained Experience')
plt.ylabel('Accuracy')
# plt.ylim(0, 1)
plt.xlim(1, NUM_EXPERIENCES)
plt.xticks(x_axis, x_axis)
plt.legend(loc='best', fontsize='small')
# plt.savefig(f'./imgs/{DATASET}_{NUM_EXPERIENCES}_{str.lower("Average_Experience_Accuracy")}.png', dpi=400)
plt.show()

In [ ]:
# PLOT 2: Average ece on all experiences after training on exp j

plt.figure(figsize=(7, 6))
x_axis = list(range(1, NUM_EXPERIENCES+1))
for i, (name, (means, stds)) in enumerate(ece_plot_data.items()):
    # print(i, name, means, stds)
    plt.plot(x_axis, means, color=plot_colors[i+1], label=name, linestyle=line_styles[i+1])
    l = [means[i] - stds[i] for i in range(len(means))]
    u = [means[i] + stds[i] for i in range(len(means))]
    plt.fill_between(x_axis, l, u, alpha=0.2, color=plot_colors[i+1])
# plt.title(f'{DATASET} - ECE')
plt.xlabel('#Trained Experience')
plt.ylabel('ECE')
# plt.ylim(0, 100)
plt.xlim(1, NUM_EXPERIENCES)
plt.xticks(x_axis, x_axis)
plt.legend(loc='best', fontsize='small')
# plt.savefig(f'./imgs/{DATASET}_{NUM_EXPERIENCES}_{str.lower("Average_Experience_ECE")}.png', dpi=400)
plt.show()

In [ ]:
plt.figure(figsize=(7, 6))
plt.plot([0, 1], [0, 1], linestyle=line_styles[0], label='ideal', color=plot_colors[0])
for i, (name, (means, stds)) in enumerate(rel_plot_data.items()):
    plt.errorbar(bins, means, yerr=stds, marker="s", linestyle=line_styles[i+1], capsize=3, capthick=1, color=plot_colors[i+1], label=name)

    # plt.plot(bins, means, color=valid_colors[i], label=name)
    # l = [means[i] - stds[i] for i in range(len(means))]
    # u = [means[i] + stds[i] for i in range(len(means))]
    # plt.fill_between(bins, l, u, alpha=0.3, color=valid_colors[i])

plt.legend(loc='best', fontsize='small')
# plt.title(f'{DATASET} - Reliability Diagram')
plt.xlabel('Accuracy')
plt.ylabel('Confidence')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.savefig(f'./imgs/{str.lower(DATASET)}_replay_reldiag.png', dpi=600)
plt.show()

In [ ]:
# HISTOGRAM : avg/std across all experiences at the end of training

# fig, axs_ = plt.subplots(2, 3, figsize=(12, 7))
fig, axs_ = plt.subplots(3, 3, figsize=(12, 10))
axs = axs_.flatten()
for i, (name, (means, stds)) in enumerate(rel_plot_data.items()):
    axs[i].plot([0, 1], [0, 1], linestyle=line_styles[0], label='ideal', color=plot_colors[0])
    # axs[i].plot(bins, m, color=valid_colors[i])
    # axs[i].fill_between(bins, l, u, alpha=0.3, linestyle='--', color=valid_colors[i])
    axs[i].errorbar(bins, means, yerr=stds, marker="s", linestyle=line_styles[1], capsize=3, capthick=1, color=plot_colors[1], label=name)
    axs[i].set_ylim(0, 1)
    axs[i].set_xlim(0, 1)
    axs[i].set_ylabel("Accuracy")
    axs[i].set_xlabel("Confidence")
    # axs[i].legend(loc='upper left', fontsize='small')
    axs[i].set_title(name)
fig.delaxes(axs_[-1][-1])
plt.tight_layout()
# TODO: updated the name of the strategy when running !!! 
# plt.savefig(f'./imgs/{str.lower(DATASET)}_all_joint_rel.png', dpi=600)
# plt.savefig(f'./imgs/{str.lower(DATASET)}_all_der_rel.png', dpi=600)
# plt.savefig(f'./imgs/{str.lower(DATASET)}_all_replay_rel.png', dpi=600)
plt.savefig(f'./imgs/{str.lower(DATASET)}_all_naive_rel.png', dpi=600)
plt.show()